In [101]:
import numpy as np
import tensorflow as tf

In [102]:
print(tf.__version__)

1.3.0


In [103]:
num_examples_per_class = 10000

classes = ['airplane','alarm clock','ambulance','angel','ant','anvil','apple','axe','banana','bandage','barn','baseball bat','baseball',
           'basket','basketball','bathtub','beach','bear','beard','bed','bee','belt','bicycle','binoculars','birthday cake','blueberry',
           'book','boomerang','bottlecap','bowtie','bracelet','brain','bread','broom','bulldozer','bus','bush','butterfly','cactus','cake',
           'calculator','calendar','camel','camera','campfire','candle','cannon','canoe','car','carrot','cello','computer',
           'cat','chandelier','clock','cloud','coffee cup','compass','cookie','couch','cow','crab','crayon','crocodile','crown',
           'cup','diamond','dog','dolphin','donut','dragon','dresser','drill','drums','duck','dumbbell','ear','elbow',
           'elephant','envelope','eraser','eye','eyeglasses','face','fan','feather','fence','finger','fire hydrant',
           'fireplace','firetruck','fish','flamingo','flashlight','flip flops','floor lamp','flower','flying saucer',
           'foot','fork']

len(classes)

100

In [104]:
x_data = np.load("./x_data_100_classes_10k.npy")

labels = [np.full((10000,), classes.index(qdraw)) for qdraw in classes]

## Concat the arrays together
y_data = np.concatenate(labels,axis=0)

# Convert to one-hot.
y_data = tf.contrib.keras.utils.to_categorical(y_data, num_classes=100)

print(x_data.shape)
print(y_data.shape)

(1000000, 784)
(1000000, 100)


In [22]:
#ds = tf.contrib.data.Dataset.from_tensor_slices??

In [23]:
#ds = tf.contrib.data.Dataset.from_tensor_slices

In [30]:
ds_x = tf.contrib.data.Dataset.from_tensor_slices(x_data)

In [25]:
ds_y = tf.contrib.data.Dataset.from_tensor_slices(y_data)

In [40]:
ds = tf.contrib.data.Dataset.from_tensor_slices((x_data,y_data))

In [41]:
ds = ds.repeat().shuffle(10000).batch(16)

In [42]:
tf.reset_default_graph()


In [43]:
next_batch = ( ds.make_one_shot_iterator().get_next())

print(next_batch)

(<tf.Tensor 'IteratorGetNext:0' shape=(?, 784) dtype=uint8>, <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=int64>)


In [44]:

with tf.Session() as sess:
    xy = sess.run(next_batch)
    


In [49]:
print(xy[0].shape)
print(xy[1].shape)

(16, 784)
(16,)


## Make the dataset for the Estimator input_fn

In [105]:
ds = tf.contrib.data.Dataset.from_tensor_slices(({'x':x_data},y_data))

In [106]:
ds = ds.repeat().shuffle(10000).batch(16)

In [107]:
tf.reset_default_graph()
next_batch = ( ds.make_one_shot_iterator().get_next())

print(next_batch)

({'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, 784) dtype=uint8>}, <tf.Tensor 'IteratorGetNext:1' shape=(?, 100) dtype=float64>)


In [108]:
with tf.Session() as sess:
    xy = sess.run(next_batch)

In [100]:
type(xy)
print(type(xy[0]['x']))
print(xy[0]['x'].shape)
print(xy[1].shape)

<class 'numpy.ndarray'>
(16, 784)
(16, 100)


In [ ]:
def train_input_fn(path):
    def input_fn():    
        dataset = (
            tf.contrib.data.TextLineDataset(path)  # create dataset from disk file
                .filter(filter_empty_lines)  # ignore empty lines
                .map(csv_decoder)  # get values on the csv row
                .shuffle(buffer_size=1000)  # shuffle the dataset, careful with the buffer_size
                .repeat()  # repeate the dataset indefinitely
                .batch(32))  # batch the data

        # create iterator
        columns = dataset.make_one_shot_iterator().get_next()
        
        # separate the label and convert it to true/false
        income = tf.equal(columns.pop('income')," >50K") 
        return columns, income
    return input_fn